# Шифрование данных страховой компании

## 1. Загрузка данных

In [100]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

data = pd.read_csv('/datasets/insurance.csv')
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [101]:
data = data.astype({"Возраст":'int64', "Зарплата":'int64'})

X = data.drop(columns='Страховые выплаты')
y = data['Страховые выплаты']

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null int64
Зарплата             5000 non-null int64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


Имеем датасет с данными о клиентах страховой компании. Все данные корректны, датасет не имеет пропусков, содержит 5000 строк и 5 признаков. Признак "Страховые выплаты" будет целевым. Заменим тип данных float64 в столбцах "Возвраст" и "Зарплата" на int64, в связи с тем, что эти признаки целочесленные.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $V$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:**

Для доказательства подставим вместо $X$ значение $XV$, тогда:

$$
w_{new} = (XV^T XV)^{-1} XV^T y\\
a_{new} = XVw
$$

Заменим в преобразованной формуле предсказания $a_{new}$ значение $w$ на $w_{new}$ и проведем преобразования:

$$ a_{new} = X V ((XV)^T XV)^{-1}(XV)^T y = \\X V (V^TX^T XV)^{-1}V^T X^T y = \\X V (X^TXV)^{-1} (V^T)^{-1} V^T X^T y = \\X V V^{-1}(X^TX)^{-1} (V^T)^{-1} V^T X^T y = \\X E (X^TX)^{-1} E X^T y = \\X (X^TX)^{-1} X^T y = X w $$

Действия:
- Раскрываем транспонирование по признаку т ранспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке ($(A B)^T = B^T A^T$)
- Допустим, что $X^TX$ - квадратная обратимая марица, $V$ - обратимая по опеределнию, то раскрываем скобки по свойству $(AB)^{-1} = B^{-1} A^{-1}$ дважды
- Пользуясь свойством обратных матриц $A A^{-1} = E$, где $E$ - это единичная матрица, ведущая себя как число $1$, преобразуем $AE = A = EA$ 
- Получаем выражение равное $w$ умноженное на $X$, что говорит о равенстве $a$ и $a_{new}$.

## 3. Алгоритм преобразования

**Алгоритм**

Найдем рандомную обратимую матрицу с ненулевым определителем, на которую потом можно будет перемножить исходные данные, соберем все эти действия в классе:

In [102]:
from sklearn.base import BaseEstimator, TransformerMixin

class NewMatrix(TransformerMixin, BaseEstimator):
    
    
    def fit(self, X, y=None, random_state=None):
        if random_state:
            np.random.seed(random_state)
            
        X_dim = X.shape[1]
        det = 0
        while det == 0:
            V = np.random.normal(size=(X_dim, X_dim))
            det = np.linalg.det(matrix)
        
        self.V = V
        self.X = X
        return self
        
    
    def transform(self, X, y=None):  
        return X.dot(self.V)

**Обоснование**

Ранее мы доказали, что при умножении исходных данных на число не влияет на качество линейной регрессии, в этом вопросе также исходим из этого.

In [104]:
X_new = NewMatrix().fit_transform(X, random_state=12345)
X.shape, X_new.shape

((5000, 4), (5000, 4))

In [105]:
X_new

,0,1,2,3
0,38224.186641,61881.000420,49961.234837,-64280.684721
1,29313.558467,47428.845564,38278.822267,-49242.555394
2,16206.481556,26215.538233,21153.670838,-27212.472653
3,32110.072445,52006.047856,42004.453110,-54044.730722
4,20126.326163,32571.440926,26289.724215,-33824.037786
...,...,...,...,...
4995,27509.697503,44537.193278,35961.967320,-46265.431844
4996,40363.894074,65360.784898,52781.234122,-67911.520319
4997,26109.730637,42282.463506,34148.283213,-43934.487817
4998,25190.905401,40790.236490,32940.675278,-42378.128295


## 4. Проверка алгоритма

In [106]:
lg = LinearRegression()

lg.fit(X, y)
predict1 = lg.predict(X)

lg.fit(X_new, y)
predict2 = lg.predict(X_new)

np.isclose(r2_score(y, predict1), r2_score(y, predict2))

True

In [107]:
np.isclose(mean_squared_error(y, predict1), mean_squared_error(y, predict2))

True

Метрики качества модели на исходных данных и на преобразованных идентичны. Что говорит о том, что при перемножении данных на матрицу данные изменяются, но качество предсказаний нет.